## Binance top 200 coins by volume and momentum

In [1]:
coins = 150

## 1- Obtengo la data usando la API de Binance

In [4]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def get_top_volume_coins_list_api(count=200):
    # Obtener la lista de todos los símbolos de trading
    symbols_url = "https://api.binance.com/api/v3/exchangeInfo"
    symbols_response = requests.get(symbols_url)
    symbols_data = symbols_response.json()
    
    # Filtrar solo los pares con USDT
    usdt_symbols = [symbol['symbol'] for symbol in symbols_data['symbols'] if symbol['symbol'].endswith('USDT')]

    # Elimino las stablecoins
    stablecoins = ['USDCUSDT', 'FDUSDUSDT', '1000SATS', '1000SATSBTC', 'WBTC','WBTCUSDT', 'WETHUSDT']
    usdt_symbols = [symbol for symbol in usdt_symbols if symbol not in stablecoins]
    
    # Calcular la fecha de inicio (7 días atrás)
    start_time = int((datetime.now() - timedelta(days=7)).timestamp() * 1000)
    
    # Obtener el volumen para cada símbolo
    volumes = []
    for symbol in usdt_symbols:
        klines_url = f"https://api.binance.com/api/v3/klines?symbol={symbol}&interval=1d&startTime={start_time}"
        klines_response = requests.get(klines_url)
        klines_data = klines_response.json()
        
        # Calcular el volumen promedio
        total_volume = sum(float(kline[7]) for kline in klines_data) #7 es quote asset volume
        avg_volume = total_volume / len(klines_data) if klines_data else 0
        volumes.append((symbol[:-4], avg_volume))  # Removemos 'USDT' del símbolo
    
    # Ordenar por volumen descendente y tomar los primeros 'count'
    top_volumes = sorted(volumes, key=lambda x: x[1], reverse=True)[:count]
    
    return top_volumes  # Ahora retornamos la lista completa de tuplas (moneda, volumen)

def get_top_volume_coins_df_api(count=200):
    volumes = get_top_volume_coins_list_api(count)
    df = pd.DataFrame(volumes, columns=['coin', 'average volume last 7 days'])

    # Ordenar el DataFrame antes de formatear los números
    df = df.sort_values('average volume last 7 days', ascending=False)

   # Formatear los números después de ordenar
    df['average volume last 7 days'] = df['average volume last 7 days'].apply(lambda x: f'{x:.2f}')

    return df

In [8]:
# Esta lista de tuplas con los volumenes se guarda en la BD
api_top_volume_coins_list = get_top_volume_coins_list_api(coins)
api_top_volume_coins_list

[('BTC', 980586168.9311401),
 ('ETH', 438517208.77666986),
 ('SOL', 317744865.1110272),
 ('NEIRO', 270918179.7951902),
 ('PEPE', 266711009.88380122),
 ('SUI', 224806355.20482144),
 ('WIF', 174604521.83472428),
 ('XRP', 115173773.46234286),
 ('BNB', 92307544.78785715),
 ('OG', 90790938.9582),
 ('DOGE', 70783648.20954286),
 ('TAO', 69502875.15836285),
 ('EIGEN', 63160911.50568286),
 ('SHIB', 62320944.192315295),
 ('SEI', 62270233.35375857),
 ('APT', 62056439.82344286),
 ('FTT', 59780656.68391572),
 ('FET', 58857238.149285726),
 ('BONK', 54124645.09039395),
 ('WLD', 48910037.61408572),
 ('FTM', 47168548.07348572),
 ('1000SATS', 43009769.67909355),
 ('RUNE', 42883423.182314284),
 ('NEAR', 41223200.95631429),
 ('SAGA', 39887572.14538429),
 ('PEOPLE', 38270113.58847014),
 ('FLOKI', 37791528.60847347),
 ('AVAX', 35284594.09538571),
 ('DOGS', 33642498.31425321),
 ('ENA', 32052978.812898856),
 ('CATI', 32023535.137119997),
 ('TRX', 31262831.35430143),
 ('NOT', 30996200.399330284),
 ('HMSTR', 30

In [5]:
#api_top_coins_df = get_top_volume_coins_df_api(coins)
#api_top_coins_df

In [9]:
# Get the ticker list for coins. sin el volumen,
top_volume_coins = [item[0] for item in api_top_volume_coins_list]
#top_volume_coins

Uso la lista manual que estoy manejando en django

In [10]:
import os

# Obtener el directorio actual
path_actual = os.getcwd()

# Construir la ruta al archivo en la carpeta 'crypto_portfolio'
ruta_archivo = os.path.join(path_actual, 'crypto_portfolio', 'nombre_del_archivo.py')

print("Ruta del archivo:", ruta_archivo)

Ruta del archivo: /Users/egrados/Library/CloudStorage/OneDrive-Personal/Github/CS50-web-programming/5-capstone/crypto_portfolio/nombre_del_archivo.py


In [11]:
from crypto_portfolio.database import tickers_list


## 2- Obtener klines para cada criptomoeda

#### Nueva version que guarda los datos de dos semanas PASADA Y ACTUAL, tanto en CSV 

- Guarda los datos en la carpeta data_past2weeks/coinBTC.csv
- NO Guarda los datos en la base de datos, en la otra version lo hace.

In [14]:
# Obtiene klines con la lista top_coins, el par BTC y el par USDT de las ultimas 2 semanas

import pandas as pd
from datetime import datetime, timedelta, timezone
import os
import requests

not_available_btcpair = []

def get_klines_btcpair_past2weeks(crypto_list):
    today = datetime.now(timezone.utc)
    
    # Calcular el lunes de la semana actual
    monday_this_week = today - timedelta(days=today.weekday())
    
    # Calcular el lunes de hace dos semanas (inicio del período)
    start_monday = monday_this_week - timedelta(weeks=2)
    start_monday_at_midnight = start_monday.replace(hour=0, minute=0, second=0, microsecond=0)
    
    # Calcular el domingo de la semana pasada (fin del período)
    end_sunday = monday_this_week - timedelta(days=1)
    end_sunday_at_midnight = end_sunday.replace(hour=23, minute=59, second=59, microsecond=999999)

    # Convertir a timestamp en UTC
    start_timestamp = str(int(start_monday_at_midnight.timestamp() * 1000))
    end_timestamp = str(int(end_sunday_at_midnight.timestamp() * 1000))

    print(f"Periodo de obtención de datos: {start_monday_at_midnight} a {end_sunday_at_midnight}")

    dataframes = []

    # Crear la carpeta 'data' si no existe
    if not os.path.exists('data_past2weeks'):
        os.makedirs('data_past2weeks')

    for crypto in crypto_list:
        print(f"Obteniendo datos para {crypto}BTC...")
        symbol = f"{crypto}BTC"

        klines_url = f"https://api.binance.com/api/v3/klines?symbol={symbol}&interval=1d&startTime={start_timestamp}&endTime={end_timestamp}"
        klines_response = requests.get(klines_url)

        if klines_response.status_code == 200:
            klines_data = klines_response.json()
            if len(klines_data) == 0:
                print(f"No hay datos disponibles para {crypto}BTC")
                not_available_btcpair.append(crypto)
                continue

            # Crear el DataFrame
            df = pd.DataFrame(klines_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
            
            # Convertir el timestamp a datetime
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            
            # Establecer el timestamp como índice
            df.set_index('timestamp', inplace=True)
            
            # Convertir las columnas necesarias a float
            for col in ['open', 'high', 'low', 'close', 'volume']:
                df[col] = df[col].astype(float)
            
            # Eliminar las columnas que no necesitamos
            df = df.drop(['close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'], axis=1)

            # Guardar el DataFrame como CSV
            filename = f"data_past2weeks/{crypto}BTC.csv"
            df.to_csv(filename)
            
            dataframes.append(df)
            print(f"Datos de {crypto}BTC guardados en {filename}")
            print('-' * 50)
        
        elif klines_response.status_code in [404, 400]:
            print(f"Error: {crypto}BTC no está disponible en Binance.")
            not_available_btcpair.append(crypto)
            print('-' * 50)

    return dataframes

In [15]:
twoweeks_klines = get_klines_btcpair_past2weeks(top_volume_coins) #dos semanas atras, rebalnce anterior


Periodo de obtención de datos: 2024-09-23 00:00:00+00:00 a 2024-10-06 23:59:59.999999+00:00
Obteniendo datos para BTCBTC...
Error: BTCBTC no está disponible en Binance.
--------------------------------------------------
Obteniendo datos para ETHBTC...
Datos de ETHBTC guardados en data_past2weeks/ETHBTC.csv
--------------------------------------------------
Obteniendo datos para SOLBTC...
Datos de SOLBTC guardados en data_past2weeks/SOLBTC.csv
--------------------------------------------------
Obteniendo datos para NEIROBTC...
Error: NEIROBTC no está disponible en Binance.
--------------------------------------------------
Obteniendo datos para PEPEBTC...
Error: PEPEBTC no está disponible en Binance.
--------------------------------------------------
Obteniendo datos para SUIBTC...
Datos de SUIBTC guardados en data_past2weeks/SUIBTC.csv
--------------------------------------------------
Obteniendo datos para WIFBTC...
Datos de WIFBTC guardados en data_past2weeks/WIFBTC.csv
-------------

## 3. Data Analysis

Ahora tengo que modificar esta funcion para que :
- Solamente haga el analisis para la semana pasada y no la antepasada. 
- Anyada la columna normalized_close. 
- Haga el calculo del slope de normalized.
- Retorne el df ordenado de mayor a menor slope normalizado.



#### Version 0: Normalizada y analiza seamana pasada y antepasada. Falla al no encontrar dato


In [14]:
# OUTDATED VERSION PARA UNA SEMANA
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

def analyze_crypto_data(data_folder='data_2'):
    results = []
    
    for filename in os.listdir(data_folder):
        if filename.endswith('.csv'):
            coin = filename.split('.')[0]
            filepath = os.path.join(data_folder, filename)
            
            # Leer el archivo CSV
            df = pd.read_csv(filepath)
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            
            # Calcular el retorno
            first_open = df.iloc[0]['open']
            last_close = df.iloc[-1]['close']
            returns = (last_close - first_open) / first_open * 100
            
            # Preparar datos para la regresión lineal
            X = np.arange(len(df)).reshape(-1, 1)
            y = df['close'].values
            
            # Realizar regresión lineal
            model = LinearRegression()
            model.fit(X, y)
            
            intercept = model.intercept_
            slope = model.coef_[0]
            
            results.append({
                'coin': coin,
                'return': returns,
                'intercept': intercept,
                'slope': slope
            })
            print(f"Análisis para {coin} completado.")
    
    # Crear DataFrame con los resultados
    result_df = pd.DataFrame(results)
    
    # Ordenar por slope de mayor a menor
    result_df = result_df.sort_values('slope', ascending=False)
    
    return result_df

# Ejemplo de uso
#df_maestro = analyze_crypto_data()
#df_maestro

#### Version 1: normalizada y analiza seamana pasada y antepasada.
Falla al no encontrar valores para la semana antepasada.

In [19]:
# Version que falla al encontrar datos faltantes, se corrige mas abajo.
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

def analyze_crypto_data_normalized(data_folder='data_past2weeks'):
    results_antepast_week = [] #results_week1
    results_past_week = [] #results_week2
    
    for filename in os.listdir(data_folder):
        if filename.endswith('.csv'):
            coin = filename.split('.')[0]
            print(f"Analizando datos de {coin}...")
            filepath = os.path.join(data_folder, filename)
            
            # Leer el archivo CSV
            df = pd.read_csv(filepath)
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            
            # Asegurarse de que el DataFrame esté ordenado por fecha
            df = df.sort_values('timestamp')
            
            # Dividir los datos en dos semanas
            past_week = df.iloc[-7:].copy()  # Semana Pasada - week2
            antepast_week = df.iloc[-14:-7].copy()  # Semana Antepasada - week1
            
            # Función para analizar una semana de datos
            def analyze_week(week_data):
                # Calcular el retorno
                first_open = week_data['open'].iloc[0]
                last_close = week_data['close'].iloc[-1]
                returns = (last_close - first_open) / first_open * 100
                
                # Normalizar los precios de cierre
                scaler = MinMaxScaler()
                week_data['normalized_close'] = scaler.fit_transform(week_data[['close']])
                
                # Preparar datos para la regresión lineal
                X = np.arange(len(week_data)).reshape(-1, 1)
                y = week_data['normalized_close'].values
                
                # Realizar regresión lineal
                model = LinearRegression()
                model.fit(X, y)
                
                intercept = model.intercept_
                slope = model.coef_[0]
                
                return {
                    'coin': coin,
                    'return': returns,
                    'intercept': intercept,
                    'slope': slope
                }
            
            # Analizar cada semana
            results_antepast_week.append(analyze_week(antepast_week))
            results_past_week.append(analyze_week(past_week))
            
            print(f"Análisis para {coin} completado.")
    
    # Crear DataFrames con los resultados
    result_df_antepast_week = pd.DataFrame(results_antepast_week)
    result_df_past_week = pd.DataFrame(results_past_week)
    
    # Ordenar por slope de mayor a menor
    result_df_antepast_week = result_df_antepast_week.sort_values('slope', ascending=False)
    result_df_past_week = result_df_past_week.sort_values('slope', ascending=False)
    
    return result_df_antepast_week, result_df_past_week

In [ ]:
# Ejemplo de uso
df_version1 = analyze_crypto_data_normalized()  #falla por las coins que no tienen data antepastweek 

#### Version 2. Normalizada y completa con 0s cuando no hay data antepastweek

In [22]:
# version que anyade 0s a la data faltante
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

def v2_analyze_crypto_data_normalized(data_folder='data_past2weeks'):
    results_antepast_week = [] #results_week1
    results_past_week = [] #results_week2
    
    for filename in os.listdir(data_folder):
        if filename.endswith('.csv'):
            coin = filename.split('.')[0]
            filepath = os.path.join(data_folder, filename)
            
            # Leer el archivo CSV
            df = pd.read_csv(filepath)
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            
            # Asegurarse de que el DataFrame esté ordenado por fecha
            df = df.sort_values('timestamp')
            
            # Verificar si hay 14 valores y completar si es necesario
            if len(df) < 14:
                last_date = df['timestamp'].max()
                missing_dates = pd.date_range(end=last_date, periods=14) - pd.Timedelta(days=13)
                missing_df = pd.DataFrame({
                    'timestamp': missing_dates,
                    'open': 0,
                    'high': 0,
                    'low': 0,
                    'close': 0,
                    'volume': 0
                })
                df = pd.concat([missing_df, df]).drop_duplicates(subset=['timestamp']).sort_values('timestamp')
                df = df.tail(14)
                print(f"Datos incompletos para {coin}. Se han rellenado los valores faltantes.")
            elif len(df) > 14:
                df = df.tail(14)
            
            # Dividir los datos en dos semanas
            past_week = df.iloc[-7:].copy()  # Semana Pasada - week2
            antepast_week = df.iloc[-14:-7].copy()  # Semana Antepasada - week1
            
            # Función para analizar una semana de datos
            def analyze_week(week_data):
                # Calcular el retorno
                first_open = week_data['open'].iloc[0]
                last_close = week_data['close'].iloc[-1]
                returns = (last_close - first_open) / first_open * 100 if first_open != 0 else 0
                
                # Normalizar los precios de cierre
                scaler = MinMaxScaler()
                week_data['normalized_close'] = scaler.fit_transform(week_data[['close']])
                
                # Preparar datos para la regresión lineal
                X = np.arange(len(week_data)).reshape(-1, 1)
                y = week_data['normalized_close'].values
                
                # Realizar regresión lineal
                model = LinearRegression()
                model.fit(X, y)
                
                intercept = model.intercept_
                slope = model.coef_[0]
                
                return {
                    'coin': coin,
                    'return': returns,
                    'intercept': intercept,
                    'slope': slope
                }
            
            # Analizar cada semana
            results_antepast_week.append(analyze_week(antepast_week))
            results_past_week.append(analyze_week(past_week))
            
            print(f"Análisis para {coin} completado.")
    
    # Crear DataFrames con los resultados
    result_df_antepast_week = pd.DataFrame(results_antepast_week)
    result_df_past_week = pd.DataFrame(results_past_week)
    
    # Ordenar por slope de mayor a menor
    result_df_antepast_week = result_df_antepast_week.sort_values('slope', ascending=False)
    result_df_past_week = result_df_past_week.sort_values('slope', ascending=False)
    
    return result_df_antepast_week, result_df_past_week

In [44]:
df_version2 = v2_analyze_crypto_data_normalized()

Análisis para MINABTC completado.
Análisis para WLDBTC completado.
Análisis para VIDTBTC completado.
Análisis para CFXBTC completado.
Análisis para BLURBTC completado.
Análisis para CRVBTC completado.
Análisis para LDOBTC completado.
Análisis para APEBTC completado.
Análisis para BNBBTC completado.
Análisis para LINKBTC completado.
Análisis para YGGBTC completado.
Análisis para ACHBTC completado.
Análisis para SUPERBTC completado.
Análisis para BNXBTC completado.
Análisis para IOBTC completado.
Análisis para OMNIBTC completado.
Análisis para XLMBTC completado.
Análisis para BANANABTC completado.
Análisis para FORTHBTC completado.
Análisis para ARKMBTC completado.
Análisis para AVAXBTC completado.
Datos incompletos para EIGENBTC. Se han rellenado los valores faltantes.
Análisis para EIGENBTC completado.
Análisis para AAVEBTC completado.
Análisis para DIABTC completado.
Análisis para ORDIBTC completado.
Análisis para ADABTC completado.
Análisis para OPBTC completado.
Análisis para UNIBTC

In [45]:
# PAST WEEK results
df_v2_pastweek = df_version2[1]
df_v2_pastweek

,coin,return,intercept,slope
36,TWTBTC,9.130982,0.141788,0.173439
97,OGBTC,52.609603,-0.166502,0.155172
72,OMBTC,16.577540,0.228754,0.146898
84,AXLBTC,0.285171,0.233742,0.144190
100,APTBTC,12.672624,0.089355,0.138445
...,...,...,...,...
77,TONBTC,-5.126767,0.926574,-0.153571
28,MKRBTC,-8.161648,0.893078,-0.160714
16,XLMBTC,-5.769231,0.865079,-0.166667
78,XRPBTC,-13.190184,0.854167,-0.171807


In [46]:
# ANTEPAST WEEK
df_v2_antepastweek = df_version2[0]
df_v2_antepastweek

,coin,return,intercept,slope
26,OPBTC,8.909774,-0.115917,0.184575
36,TWTBTC,23.341140,0.030712,0.182462
66,DOGEBTC,13.095238,-0.097143,0.177143
82,ZROBTC,15.454545,0.094829,0.174407
52,PEOPLEBTC,22.000000,0.004870,0.173701
...,...,...,...,...
75,DYMBTC,1.213682,1.046311,-0.145212
89,VANRYBTC,3.164557,0.942308,-0.145604
74,TIABTC,8.907363,0.868112,-0.147194
8,BNBBTC,-1.944264,0.878205,-0.159035


In [36]:
#Valido que efectivamente este colocando 0 en los valores que fallaron anteriormente.
test =  df_v2_antepastweek[df_v2_antepastweek['coin'] == "EIGENBTC"]
test

,coin,return,intercept,slope
21,EIGENBTC,0.0,0.0,0.0


#### Version 3. Normalizada y completa con 0s cuando no hay data antepastweek. Añade R2 como parametro y filtra unicamente los que tienen R2 mayor a 0.9.

In [38]:
# VERSION QUE USA R2
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

def v3_analyze_crypto_data_normalized(data_folder='data_past2weeks'):
    results_antepast_week = [] #results_week1
    results_past_week = [] #results_week2
    
    for filename in os.listdir(data_folder):
        if filename.endswith('.csv'):
            coin = filename.split('.')[0]
            filepath = os.path.join(data_folder, filename)
            
            # Leer el archivo CSV
            df = pd.read_csv(filepath)
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            
            # Asegurarse de que el DataFrame esté ordenado por fecha
            df = df.sort_values('timestamp')
            
            # Verificar si hay 14 valores y completar si es necesario
            if len(df) < 14:
                last_date = df['timestamp'].max()
                missing_dates = pd.date_range(end=last_date, periods=14) - pd.Timedelta(days=13)
                missing_df = pd.DataFrame({
                    'timestamp': missing_dates,
                    'open': 0,
                    'high': 0,
                    'low': 0,
                    'close': 0,
                    'volume': 0
                })
                df = pd.concat([missing_df, df]).drop_duplicates(subset=['timestamp']).sort_values('timestamp')
                df = df.tail(14)
                print(f"Datos incompletos para {coin}. Se han rellenado los valores faltantes.")
            elif len(df) > 14:
                df = df.tail(14)
            
            # Dividir los datos en dos semanas
            past_week = df.iloc[-7:].copy()  # Semana Pasada - week2
            antepast_week = df.iloc[-14:-7].copy()  # Semana Antepasada - week1
            
            # Función para analizar una semana de datos
            def analyze_week(week_data):
                # Calcular el retorno
                first_open = week_data['open'].iloc[0]
                last_close = week_data['close'].iloc[-1]
                returns = (last_close - first_open) / first_open * 100 if first_open != 0 else 0
                
                # Normalizar los precios de cierre
                scaler = MinMaxScaler()
                week_data['normalized_close'] = scaler.fit_transform(week_data[['close']])
                
                # Preparar datos para la regresión lineal
                X = np.arange(len(week_data)).reshape(-1, 1)
                y = week_data['normalized_close'].values
                
                # Realizar regresión lineal
                model = LinearRegression()
                model.fit(X, y)
                
                intercept = model.intercept_
                slope = model.coef_[0]
                
                # Calcular R2
                r2 = r2_score(y, model.predict(X))
                
                return {
                    'coin': coin,
                    'return': returns,
                    'intercept': intercept,
                    'slope': slope,
                    'r2': r2
                }
            
            # Analizar cada semana
            results_antepast_week.append(analyze_week(antepast_week))
            results_past_week.append(analyze_week(past_week))
            
            print(f"Análisis para {coin} completado.")
    
    # Crear DataFrames con los resultados
    result_df_antepast_week = pd.DataFrame(results_antepast_week)
    result_df_past_week = pd.DataFrame(results_past_week)
    
    # Filtrar y ordenar los resultados a los que tienen R2 >= 0.9
    def filter_and_sort(df):
        df_filtered = df[df['r2'] >= 0.9]
        return df_filtered.sort_values('slope', ascending=False)
    
    result_df_antepast_week = filter_and_sort(result_df_antepast_week)
    result_df_past_week = filter_and_sort(result_df_past_week)
    
    # Asegurarse de que R2 esté incluido en los resultados finales
    columns_order = ['coin', 'return', 'slope', 'intercept', 'r2']
    result_df_antepast_week = result_df_antepast_week[columns_order]
    result_df_past_week = result_df_past_week[columns_order]
    
    return result_df_antepast_week, result_df_past_week

In [39]:
df_version3 = v3_analyze_crypto_data_normalized()

Análisis para MINABTC completado.
Análisis para WLDBTC completado.
Análisis para VIDTBTC completado.
Análisis para CFXBTC completado.
Análisis para BLURBTC completado.
Análisis para CRVBTC completado.
Análisis para LDOBTC completado.
Análisis para APEBTC completado.
Análisis para BNBBTC completado.
Análisis para LINKBTC completado.
Análisis para YGGBTC completado.
Análisis para ACHBTC completado.
Análisis para SUPERBTC completado.
Análisis para BNXBTC completado.
Análisis para IOBTC completado.
Análisis para OMNIBTC completado.
Análisis para XLMBTC completado.
Análisis para BANANABTC completado.
Análisis para FORTHBTC completado.
Análisis para ARKMBTC completado.
Análisis para AVAXBTC completado.
Datos incompletos para EIGENBTC. Se han rellenado los valores faltantes.
Análisis para EIGENBTC completado.
Análisis para AAVEBTC completado.
Análisis para DIABTC completado.
Análisis para ORDIBTC completado.
Análisis para ADABTC completado.
Análisis para OPBTC completado.
Análisis para UNIBTC

In [42]:
# PAST WEEK
df_v3_antepastweek = df_version3[1]
df_v3_antepastweek

,coin,return,slope,intercept,r2
28,MKRBTC,-8.161648,-0.160714,0.893078,0.931503


In [43]:
#ANTEPAST WEEK
df_v2_pastweek = df_version3[0]

df_v2_pastweek

,coin,return,slope,intercept,r2
87,WIFBTC,40.300188,0.152571,0.114726,0.959441
21,EIGENBTC,0.000000,0.000000,0.000000,1.000000
13,BNXBTC,-16.569921,-0.177644,1.011596,0.963475


#### Version 4. Normalizada y completa con 0s cuando no hay data antepastweek. Añade R2 como parametro y NO filtra  los que tienen R2 mayor a 0.9.

In [47]:
#pendiente de hacer 

## 4. Ploteo de grafica

In [22]:
# Ploteo 1, plotea bien el linechart pero no el candlestick

import os
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf

def plot_crypto_data(df_maestro, data_folder='data', plot_folder='plots'):
    # Crear la carpeta 'plots' si no existe
    if not os.path.exists(plot_folder):
        os.makedirs(plot_folder)

    #for _, row in df_maestro.iterrows():
    for _, row in df_maestro.head(6).iterrows():
        coin = row['coin']
        slope = row['slope']
        intercept = row['intercept']

        # Leer el archivo CSV correspondiente
        df = pd.read_csv(os.path.join(data_folder, f"{coin}.csv"))
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df.set_index('timestamp', inplace=True)

        # Calcular la línea de regresión
        x = range(len(df))
        regression_line = [slope * xi + intercept for xi in x]

        # Gráfico de línea
        plt.figure(figsize=(12, 6))
        plt.plot(df.index, df['close'], label='Precio de cierre')
        plt.plot(df.index, regression_line, label='Regresión lineal', color='red')
        plt.title(f"Análisis de {coin} - Gráfico de línea")
        plt.xlabel("Fecha")
        plt.ylabel("Precio")
        plt.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(os.path.join(plot_folder, f"{coin}_line.png"))
        plt.show()

        # Gráfico de velas (candlestick)
        fig, ax = plt.subplots(figsize=(12, 6))
        mpf.plot(df, type='candle', style='yahoo', ax=ax)
        ax.plot(df.index, regression_line, label='Regresión lineal', color='red')
        ax.set_title(f"Análisis de {coin} - Gráfico de velas")
        ax.set_xlabel("Fecha")
        ax.set_ylabel("Precio")
        ax.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(os.path.join(plot_folder, f"{coin}_candlestick.png"))
        plt.show()

# Ejemplo de uso:
#df_maestro = analyze_crypto_data()  # Asumiendo que esta función ya fue definida y ejecutada
plot_crypto_data(df_maestro)

NameError: name 'df_maestro' is not defined

In [47]:
asd = df_maestro.iterrows()
for _, row in df_maestro.head(10).iterrows():
    print (row['coin'])

TAOBTC
ETHBTC
AAVEBTC
BNBBTC
BCHBTC
SOLBTC
TRBBTC
METISBTC
AVAXBTC
ORDIBTC
